In [ ]:
import torch
import torch.nn as nn
import torchvision 
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((225,225)),
    torchvision.transforms.CenterCrop((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean = [0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

In [ ]:
root = './cifar_data/'

# Create the directory if it doesn't exist
if not os.path.exists(root):
    os.makedirs(root)

In [ ]:
batch_size = 32
num_epoch = 10
learning_rate = 0.01
momentum = 0.9

In [ ]:
train_data = torchvision.datasets.CIFAR100(root=root, download = True, train = True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size = batch_size, num_workers = 4)
test_data = torchvision.datasets.CIFAR100(root=root, download = True, train = False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size = batch_size, num_workers = 4)

In [ ]:
class block(nn.Module):
  def __init__(self, in_channels, out_channels, identity_downsample = None, stride = 1):
    super(block, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
    self.bn1 =  nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
    self.bn2 =  nn.BatchNorm2d(out_channels)
    self.conv3 = nn.Conv2d(out_channels, out_channels*4, kernel_size=1, stride=1, padding=0)
    self.bn3 =  nn.BatchNorm2d(out_channels*4)

    self.relu = nn.ReLU()
    self.identity_downsample = identity_downsample

  def forward(self,x):
    identity = x

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.conv3(x)
    x = self.bn3(x)

    if self.identity_downsample is not None:
      identity = self.identity_downsample(identity)

    x += identity
    x = self.relu(x)
    return x


class ResNet(nn.Module):
  def __init__(self, block, image_channels, num_classes, layers):
    super(ResNet, self).__init__()
    self.in_channels = 64
    self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    # Resnet Layers
    self.layer1 = self._make_layer(block, layers[0], self.in_channels, 64, stride=1)
    self.layer2 = self._make_layer(block, layers[1], self.in_channels, 128, stride=2)
    self.layer3 = self._make_layer(block, layers[2], self.in_channels, 256, stride=2)
    self.layer4 = self._make_layer(block, layers[3], self.in_channels, 512, stride=2)

    self.adaptiveavgpool =  nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512*4, num_classes)

  def forward(self,x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.max_pool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    
    x =  self.adaptiveavgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x)

    return x

  def _make_layer(self, block,num_residual_blocks, in_channels, out_channels, stride):
    identity_downsample = None
    layers = []

    if stride != 1 or self.in_channels != out_channels*4:
      identity_downsample = nn.Sequential(nn.Conv2d(in_channels, out_channels*4, kernel_size=1, stride=stride, padding=0), nn.BatchNorm2d(out_channels*4))

    layers.append(block(in_channels, out_channels, identity_downsample, stride))
    self.in_channels = out_channels * 4 

    for i in range(num_residual_blocks-1):
      layers.append(block(self.in_channels, out_channels))

    return nn.Sequential(*layers)   

In [ ]:
def ResNet50(image_channels = 3, num_classes=100):
  return ResNet(block, image_channels, num_classes, [3,4,6,3])

In [ ]:
model = ResNet50()
model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate, momentum=momentum )

In [ ]:
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        predicted = model(images)
        loss = loss_func(predicted, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f"[{epoch}, {i+1}], loss : {loss}")

In [ ]:
model.eval()

In [ ]:
total = correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        predict = model(images)
        predcited_softmax = nn.Softmax(1)(predict)
        _, output = torch.max(predcited_softmax, dim=1)
        correct += (output == labels).sum().item()
        total += labels.size(0)
    
    print(f"Accuracy of the Model : {100*(correct/total)}%")